In [15]:
import pandas as pd
import numpy as np
import wrangle_zillow
import acquire
import scipy.stats as stats

In [2]:
df = acquire.get_zillow_data()

In [3]:
df.shape

(77580, 69)

In [4]:
df['parcelid'].nunique()

77381

In [5]:
# Dropping duplicate parcelid's and storing in to a new dataframe
df1 = df.sort_values(by='transactiondate', ascending=True).drop_duplicates(subset='parcelid',keep='last')

In [6]:
df1.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 77381 entries, 0 to 77579
Data columns (total 69 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   typeconstructiontypeid        222 non-null    float64
 1   storytypeid                   50 non-null     float64
 2   heatingorsystemtypeid         49440 non-null  float64
 3   buildingclasstypeid           15 non-null     float64
 4   architecturalstyletypeid      206 non-null    float64
 5   airconditioningtypeid         24953 non-null  float64
 6   propertylandusetypeid         77381 non-null  float64
 7   parcelid                      77381 non-null  int64  
 8   id                            77381 non-null  int64  
 9   basementsqft                  50 non-null     float64
 10  bathroomcnt                   77381 non-null  float64
 11  bedroomcnt                    77381 non-null  float64
 12  buildingqualitytypeid         49672 non-null  float64
 13  c

In [7]:
df1.shape

(77381, 69)

In [8]:
df1.nunique()

typeconstructiontypeid       4
storytypeid                  1
heatingorsystemtypeid       10
buildingclasstypeid          2
architecturalstyletypeid     5
                            ..
architecturalstyledesc       5
buildingclassdesc            2
heatingorsystemdesc         10
storydesc                    1
typeconstructiondesc         4
Length: 69, dtype: int64

In [9]:
def nulls_by_col(df):
    num_missing = df.isnull().sum()
    rows = df.shape[0]
    prcnt_miss = num_missing / rows * 100
    cols_missing = pd.DataFrame({'num_rows_missing': num_missing, 'percent_rows_missing': prcnt_miss})
    return cols_missing

def nulls_by_row(df):
    num_missing = df.isnull().sum(axis=1)
    prcnt_miss = num_missing / df.shape[1] * 100
    rows_missing = pd.DataFrame({'num_cols_missing': num_missing, 'percent_cols_missing': prcnt_miss})\
    .reset_index()\
    .groupby(['num_cols_missing', 'percent_cols_missing']).count()\
    .rename(index=str, columns={'index': 'num_rows'}).reset_index()
    return rows_missing

In [10]:
nulls_by_col(df1)

,num_rows_missing,percent_rows_missing
typeconstructiontypeid,77159,99.713108
storytypeid,77331,99.935385
heatingorsystemtypeid,27941,36.108347
buildingclasstypeid,77366,99.980615
architecturalstyletypeid,77175,99.733785
...,...,...
architecturalstyledesc,77175,99.733785
buildingclassdesc,77366,99.980615
heatingorsystemdesc,27941,36.108347
storydesc,77331,99.935385


In [11]:
nulls_by_row(df1)

,num_cols_missing,percent_cols_missing,num_rows
0,23,33.33333333333333,2
1,24,34.78260869565217,13
2,25,36.231884057971016,24
3,26,37.68115942028986,65
4,27,39.130434782608695,316
5,28,40.57971014492754,455
6,29,42.028985507246375,5270
7,30,43.47826086956522,3455
8,31,44.927536231884055,9891
9,32,46.3768115942029,12579


In [12]:
def summarize(df):
    '''
    This function will take in a single argument (pandas DF)
    and output to console various statistics on said DF, including:
    # .head()
    # .info()
    # .describe()
    # value_counts()
    # observe null values
    '''
    print('----------------------------------------------------')
    print('DataFrame Head')
    print(df.head(3))
    print('----------------------------------------------------')
    print('DataFrame Info')
    print(df.info())
    print('----------------------------------------------------')
    print('DataFrame Description')
    print(df.describe())
    num_cols = df.select_dtypes(exclude='O').columns.to_list()
    cat_cols = df.select_dtypes(include='O').columns.to_list()
    print('----------------------------------------------------')
    print('DataFrame Value Counts: ')
    for col in df.columns:
        if col in cat_cols:
            print(df[col].value_counts())
        else:
            print(df[col].value_counts(bins=10, sort=False))
    print('----------------------------------------------------')
    print('Nulls in DataFrame by Column: ')
    print(nulls_by_col(df))
    print('----------------------------------------------------')
    print('Nulls in DataFrame by Rows: ')
    print(nulls_by_row(df))
    print('----------------------------------------------------')

In [13]:
summarize(df1)

----------------------------------------------------
DataFrame Head
   typeconstructiontypeid  storytypeid  heatingorsystemtypeid  \
0                     NaN          NaN                    NaN   
1                     NaN          NaN                    NaN   
2                     NaN          NaN                    NaN   

   buildingclasstypeid  architecturalstyletypeid  airconditioningtypeid  \
0                  NaN                       NaN                    NaN   
1                  NaN                       NaN                    NaN   
2                  NaN                       NaN                    NaN   

   propertylandusetypeid  parcelid       id  basementsqft  ...  id.1  \
0                  261.0  14297519  1727539           NaN  ...     0   
1                  261.0  17052889  1387261           NaN  ...     1   
2                  261.0  14186244    11677           NaN  ...     2   

   logerror  transactiondate        propertylandusedesc  airconditioningdesc  \
0

(0.988, 2.1]       35
(2.1, 3.2]        329
(3.2, 4.3]       9483
(4.3, 5.4]       2088
(5.4, 6.5]      13462
(6.5, 7.6]       5971
(7.6, 8.7]      14645
(8.7, 9.8]       2299
(9.8, 10.9]       313
(10.9, 12.0]     1047
Name: buildingqualitytypeid, dtype: int64
(0.982, 2.7]    52390
(2.7, 4.4]      21699
(4.4, 6.1]       2361
(6.1, 7.8]        176
(7.8, 9.5]        134
(9.5, 11.2]        10
(11.2, 12.9]        0
(12.9, 14.6]        1
(14.6, 16.3]        0
(16.3, 18.0]        1
Name: calculatedbathnbr, dtype: int64
(65.93299999999999, 65.947]      0
(65.947, 65.96]                  0
(65.96, 65.974]                  0
(65.974, 65.987]                 0
(65.987, 66.0]                 614
(66.0, 66.013]                   0
(66.013, 66.026]                 0
(66.026, 66.04]                  0
(66.04, 66.053]                  0
(66.053, 66.066]                 0
Name: decktypeid, dtype: int64
(37.131, 730.8]      702
(730.8, 1417.6]     3025
(1417.6, 2104.4]    1733
(2104.4, 2791.2]     336

(-0.016, 1.5]    59809
(1.5, 3.0]         226
(3.0, 4.5]        1415
(4.5, 6.0]        7529
(6.0, 7.5]        3838
(7.5, 9.0]        4065
(9.0, 10.5]        339
(10.5, 12.0]       143
(12.0, 13.5]        10
(13.5, 15.0]         7
Name: roomcnt, dtype: int64
(0.993, 1.6]    10012
(1.6, 2.2]         75
(2.2, 2.8]          0
(2.8, 3.4]          8
(3.4, 4.0]          0
(4.0, 4.6]          0
(4.6, 5.2]          0
(5.2, 5.8]          0
(5.8, 6.4]          0
(6.4, 7.0]          1
Name: threequarterbathnbr, dtype: int64
(0.763, 24.6]     50560
(24.6, 48.2]          2
(48.2, 71.8]          0
(71.8, 95.4]          0
(95.4, 119.0]         0
(119.0, 142.6]        0
(142.6, 166.2]        0
(166.2, 189.8]        0
(189.8, 213.4]        0
(213.4, 237.0]        1
Name: unitcnt, dtype: int64
(7.819, 329.0]      1635
(329.0, 647.0]       607
(647.0, 965.0]        95
(965.0, 1283.0]       34
(1283.0, 1601.0]       9
(1601.0, 1919.0]       1
(1919.0, 2237.0]       2
(2237.0, 2555.0]       2
(2555.0, 2873.

   num_cols_missing percent_cols_missing  num_rows
0                23    33.33333333333333         2
1                24    34.78260869565217        13
2                25   36.231884057971016        24
3                26    37.68115942028986        65
4                27   39.130434782608695       316
5                28    40.57971014492754       455
6                29   42.028985507246375      5270
7                30    43.47826086956522      3455
8                31   44.927536231884055      9891
9                32     46.3768115942029     12579
10               33    47.82608695652174     14782
11               34   49.275362318840585     13327
12               35    50.72463768115942      5148
13               36    52.17391304347826      5775
14               37    53.62318840579711      3620
15               38   55.072463768115945      1925
16               39    56.52173913043478       285
17               40   57.971014492753625       230
18               41    59.42028

In [14]:
df1.dtypes

typeconstructiontypeid      float64
storytypeid                 float64
heatingorsystemtypeid       float64
buildingclasstypeid         float64
architecturalstyletypeid    float64
                             ...   
architecturalstyledesc       object
buildingclassdesc            object
heatingorsystemdesc          object
storydesc                    object
typeconstructiondesc         object
Length: 69, dtype: object